In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime as dt
import feltsson_lib as flt

In [2]:
#Новый список
for filename in os.listdir('./inputsson'):
    if filename.endswith('.csv'):
        df = pd.read_csv('./inputsson/' + filename, sep=';', encoding='cp1251', dtype=({'INN':str, 'DT':str}), error_bad_lines=False, warn_bad_lines=True)
df=df.drop(columns=['Unnamed: 9'])
df.columns = ['Дата', 'Название', 'ИНН', 'Адрес', 'Сайт', 'Признаки, установленные Банком России', 'Регионы', 'Деятельность прекращена', 'Дополнительно']

In [3]:
#Cтарый список
for filename in os.listdir('./inputsson/old'):
    if filename.endswith('.csv'):
        df_old = pd.read_csv('./inputsson/old/' + filename, sep=';', encoding='cp1251', dtype=({'INN':str, 'DT':str}), error_bad_lines=False, warn_bad_lines=True)
df_old=df_old.drop(columns=['Unnamed: 9'])
df_old.columns = ['Дата', 'Название', 'ИНН', 'Адрес', 'Сайт', 'Признаки, установленные Банком России', 'Регионы', 'Деятельность прекращена', 'Дополнительно']

In [4]:
#Привоодим к формату даты и определяем последнюю загруженную дату
df['Дата'] = df['Дата'].apply(lambda x: dt.datetime.strptime(x, '%d.%m.%Y'))
df_old['Дата'] = df_old['Дата'].apply(lambda x: dt.datetime.strptime(x, '%d.%m.%Y'))
last_date = max(df_old['Дата'])

In [ ]:
#Сверка старого и свежего списков ЦБ на предмет удаленных ИНН
df_new = df.loc[df['Дата'] <= last_date]
df_INN_del = df_old.loc[~df_old['ИНН'].isin(df_new['ИНН']), 'ИНН']
df_INN_del.to_csv('202_INN_DEL.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
#датафрейм с новыми записями
df=df.loc[df['Дата'] > last_date]

In [ ]:
# Список ИНН
df_INN = df['ИНН'].dropna().reset_index(drop = True)
df_INN.to_csv('202_INN.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
# Список названий мерчантов с наличием адресов
df_address = df[['Адрес', 'Название']]
df_address = df_address.loc[pd.notnull(df_address['Адрес'])]
# Разбиваем названия по запятым и |
df_name = df_address['Название'].str.split(', | \| ', expand=True)
# Сводим с адресами
df_address = pd.concat([df_address['Адрес'], df_name], axis = 1)
df_address = df_address.set_index('Адрес').stack().reset_index(name='Имя').drop('level_1',axis='columns')

In [ ]:
# Получаем значения имен из кавычек
flt.quotes(df_address['Имя'])
#Транслитерируем имена мерчантов
df_address['Имя'] = df_address['Имя'].apply(lambda x: flt.tr(x) if re.search('[А-ЯЁ]{4}', x) else flt.tr2(x))

In [ ]:
# Вставляем столбцы Индекс и Город
df_address.insert(1, 'Индекс', None)
df_address.insert(2, 'Город', 'FILL ' + df_address['Адрес'])
# Получаем значения индексов
for i in range(df_address['Адрес'].shape[0]):
    row = df_address['Адрес'].iloc[i]
    if re.match('(\d{6}),', row):
        df_address['Индекс'].iloc[i] = re.match('^(\d{6}),', row).group(1)
# Получаем значения городов и сел
for i in range(df_address['Адрес'].shape[0]):
    row = df_address['Адрес'].iloc[i]
    if re.search(',\s{1,}[^\.]*[г|с|п|д|Г|С|П|Д]\.\s([^,\d\.]+),', row):
        df_address['Город'].iloc[i] = re.search(',\s{1,}[^\.]*[г|с|п|д|Г|С|П|Д]\.\s([^,\d\.]+),', row).group(1)
# Транслитерируем города
df_address['Город'] = df_address['Город'].apply(lambda x: flt.tr(x) if re.search('[А-ЯЁ]{4}', x) else flt.tr2(x))
#Удаляем дубликаты
df_address = df_address.drop_duplicates().reset_index(drop=True)

In [ ]:
# Имена мерчантов без условий на адрес и индекс, не вошедшие в предыдущий список, и все сайты
df_names = df[~df['Адрес'].isin(df_address['Адрес'])]
df_names = pd.concat([df_names['Название'], df['Сайт'].loc[pd.notnull(df['Сайт'])]], axis = 0)
df_names = df_names.str.split(', | \| ', expand=True)
df_names = df_names.stack().reset_index(name='Имя').drop(['level_0', 'level_1'],axis='columns')
# Получаем значения имен из кавычек
flt.quotes(df_names['Имя'])
# Транслитерируем имена мерчантов
df_names['Имя'] = df_names['Имя'].apply(lambda x: flt.tr(x) if re.search('[А-ЯЁ]{4}', x) else flt.tr2(x))
#Убираем домены из сайтов
df_names['Имя'] = df_names['Имя'].str.replace('[W{3}|w{3}]\.', '', regex = True)
df_names['Имя'] = df_names['Имя'].str.replace('\.\w{,4}$', '', regex = True)
#Удаляем дубликаты
df_names = df_names.drop_duplicates().reset_index(drop=True)

In [ ]:
#Убираем "мусорные" слова
df_trash = pd.read_excel('TrashWords.xlsx', header = None, index_col = None, dtype = str)
for word in df_trash[0]:
    df_names['Имя'] = df_names['Имя'].str.replace(word, '', regex = False)
    df_address['Имя'] = df_address['Имя'].str.replace(word, '', regex = False)

In [ ]:
#Убираем образовавшиеся пустые строки
df_names = df_names[df_names['Имя'] != '']
df_address = df_address[df_address['Имя'] != '']

In [ ]:
#Убираем символы переноса строки, пробелы, кавычки и скобки
df_names['Имя'] = df_names['Имя'].str.strip('\n\r \"\(\)')
df_address['Имя'] = df_address['Имя'].str.strip('\n\r \"\(\)\(\)')
df_address['Город'] = df_address['Город'].str.strip('\n\r \"\(\)')
df_address['Индекс'] = df_address['Индекс'].str.strip('\n\r \"\(\)')

In [ ]:
#Делаем списки для ручной корректировки
df_names.to_csv('201_Names_EPOS.csv', sep=';', encoding='cp1251', index=False)
df_address.drop(columns='Адрес').to_csv('203_205_Index_Cities.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
with open("Журнал обновления списков.txt", "a") as file:
    file.write('Последняя запись в списке ЦБ добавлена ' + df['Дата'].max().strftime('%d.%m.%Y') + '\n')